In [22]:
# import library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import plotly
import sklearn
import re
import random
import math


In [77]:
# Separate data into training and validation
total_num=pd.date_range(start='1/1/2018',
                        end='12/31/2018',
                        freq='D')
# Set seed in order to attain reproducibility
length=len(total_num)
random.seed(498)
total_num=pd.Series(total_num)
lst=[]
for date in total_num:
    date=str(date).split()[0]
    date=date.replace('-','_')
    lst.append(date)
total_num=pd.Series(lst)

# Choice the training date
training_num=random.sample(list(np.arange(0,365)),math.floor(length*5/6),)
remaining_len=length-len(training_num)
validation_num=total_num.drop(index=training_num)

In [78]:
print(training_num)

[246, 212, 312, 126, 127, 189, 195, 7, 306, 364, 304, 134, 349, 65, 325, 313, 278, 230, 231, 243, 351, 184, 282, 80, 54, 158, 207, 279, 280, 342, 203, 114, 121, 321, 335, 156, 34, 129, 104, 336, 160, 181, 263, 186, 174, 201, 154, 39, 215, 61, 247, 209, 208, 3, 49, 122, 317, 256, 74, 131, 31, 363, 144, 13, 25, 197, 167, 350, 185, 251, 143, 262, 109, 152, 314, 332, 89, 259, 327, 21, 165, 345, 205, 116, 242, 223, 6, 149, 63, 338, 331, 120, 101, 229, 95, 47, 241, 136, 347, 4, 339, 281, 38, 360, 254, 257, 148, 255, 307, 17, 124, 44, 202, 26, 297, 219, 35, 190, 88, 291, 348, 236, 132, 337, 199, 178, 354, 123, 110, 179, 237, 77, 118, 20, 214, 22, 78, 42, 76, 272, 277, 84, 5, 16, 187, 298, 355, 283, 28, 15, 169, 113, 159, 58, 92, 140, 33, 361, 43, 261, 340, 296, 326, 222, 191, 323, 264, 19, 292, 194, 103, 173, 108, 170, 139, 353, 105, 193, 86, 258, 226, 192, 268, 27, 318, 93, 227, 238, 112, 50, 198, 40, 244, 36, 253, 87, 9, 206, 221, 176, 188, 248, 111, 250, 228, 252, 218, 269, 142, 11, 295, 7

In [79]:
print(validation_num)

0      2018_01_01
1      2018_01_02
2      2018_01_03
41     2018_02_11
46     2018_02_16
          ...    
346    2018_12_13
352    2018_12_19
357    2018_12_24
359    2018_12_26
362    2018_12_29
Length: 61, dtype: object


In [60]:
# Read data
file='./AIS_2019_01_01.csv'
df=pd.read_csv(file)

In [81]:
# Delete VesselType NA value
df.head()
df=df[df['VesselType'].isna()==False]

In [83]:
df.head()
df.describe()

,MMSI,LAT,LON,SOG,COG,Heading,VesselType,Status,Length,Width,Draft,Cargo
count,7.039789e+06,7.039789e+06,7.039789e+06,7.039789e+06,7.039789e+06,7.039789e+06,7.039789e+06,5.634791e+06,6.525718e+06,5.488049e+06,1.949847e+06,2.530337e+06
mean,3.672707e+08,3.525674e+01,-9.586362e+01,1.823239e+00,-2.550855e+01,3.691049e+02,4.376085e+01,3.975022e+00,5.007536e+01,1.160537e+01,6.521161e+00,4.752748e+01
std,5.362156e+07,8.248943e+00,2.005094e+01,3.907017e+00,1.223710e+02,1.784671e+02,2.000086e+01,5.750609e+00,6.591141e+01,9.417119e+00,4.349750e+00,2.246778e+01
min,0.000000e+00,6.667000e-02,-1.769794e+02,-5.110000e+01,-2.048000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.250000e+01,0.000000e+00
25%,3.669418e+08,2.962486e+01,-9.706897e+01,0.000000e+00,-1.318000e+02,2.150000e+02,3.100000e+01,0.000000e+00,1.800000e+01,7.000000e+00,3.400000e+00,3.100000e+01
50%,3.673522e+08,3.251481e+01,-9.027533e+01,0.000000e+00,-4.960000e+01,5.110000e+02,3.100000e+01,0.000000e+00,2.500000e+01,9.000000e+00,4.700000e+00,5.200000e+01
75%,3.676476e+08,4.053016e+01,-8.151780e+01,6.000000e-01,8.100000e+01,5.110000e+02,6.000000e+01,5.000000e+00,4.100000e+01,1.200000e+01,9.800000e+00,6.000000e+01
max,1.000000e+09,7.049377e+01,-6.087994e+01,5.080000e+01,2.047000e+02,5.110000e+02,1.700000e+02,1.500000e+01,3.970000e+02,1.260000e+02,2.410000e+01,1.700000e+02


In [159]:
# Group by VesselType 
vessel_list=list(df['VesselType'].unique())
MMSI_list=pd.Series(df['MMSI'].unique())
VesselType=30
df1=df[df['VesselType']==VesselType]
df1.head()

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TranscieverClass
16,368066040,2019-01-01T00:00:05,30.33668,-88.29042,6.6,201.1,202.0,BOSSMAN,IMO0000000,WDK5560,30.0,NaN,30.0,9.0,NaN,NaN,B
24,367138430,2019-01-01T00:00:06,38.95835,-74.86710,0.0,-140.1,511.0,AMY MARIE,IMO7932111,WUS2556,30.0,7.0,27.0,7.0,NaN,30.0,B
56,367776890,2019-01-01T00:00:04,29.41973,-84.90810,2.9,-191.2,511.0,LONI LYNN,NaN,WDJ4354,30.0,NaN,19.0,NaN,NaN,NaN,B
75,366896490,2019-01-01T00:00:05,47.65940,-122.38152,0.0,187.4,511.0,AMERICAN LADY,IMO7513006,WDB4259,30.0,0.0,27.0,8.0,NaN,NaN,B
112,367046940,2019-01-01T00:00:06,47.65615,-122.36628,0.0,154.0,125.0,GLADIATOR,IMO7938115,WCZ9000,30.0,0.0,33.0,9.0,NaN,0.0,B


In [91]:
MMSI_list=pd.Series(df1['MMSI'].unique())
MMSI_list.head()

0    368066040
1    367138430
2    367776890
3    366896490
4    367046940
dtype: int64

In [160]:
MMSI=368066040
df1=df1.loc[df1['MMSI']==MMSI]
df1.head(1000)


,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TranscieverClass
16,368066040,2019-01-01T00:00:05,30.33668,-88.29042,6.6,201.1,202.0,BOSSMAN,IMO0000000,WDK5560,30.0,NaN,30.0,9.0,NaN,NaN,B
10261,368066040,2019-01-01T00:01:35,30.33413,-88.29156,6.7,201.4,203.0,BOSSMAN,IMO0000000,WDK5560,30.0,NaN,30.0,9.0,NaN,NaN,B
17861,368066040,2019-01-01T00:03:04,30.33155,-88.29278,6.6,203.9,205.0,BOSSMAN,IMO0000000,WDK5560,30.0,NaN,30.0,9.0,NaN,NaN,B
28808,368066040,2019-01-01T00:05:05,30.32804,-88.29416,6.6,197.9,198.0,BOSSMAN,IMO0000000,WDK5560,30.0,NaN,30.0,9.0,NaN,NaN,B
39174,368066040,2019-01-01T00:06:34,30.32549,-88.29514,5.9,200.5,203.0,BOSSMAN,IMO0000000,WDK5560,30.0,NaN,30.0,9.0,NaN,NaN,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4632518,368066040,2019-01-01T15:16:24,29.44726,-88.50353,3.3,-163.2,253.0,BOSSMAN,IMO0000000,WDK5560,30.0,NaN,30.0,9.0,NaN,NaN,B
4701028,368066040,2019-01-01T15:28:55,29.44257,-88.51486,3.2,-169.1,245.0,BOSSMAN,IMO0000000,WDK5560,30.0,NaN,30.0,9.0,NaN,NaN,B
4882957,368066040,2019-01-01T16:02:24,29.41620,-88.52515,3.5,148.4,148.0,BOSSMAN,IMO0000000,WDK5560,30.0,NaN,30.0,9.0,NaN,NaN,B
4887321,368066040,2019-01-01T16:00:53,29.41742,-88.52550,3.1,170.3,164.0,BOSSMAN,IMO0000000,WDK5560,30.0,NaN,30.0,9.0,NaN,NaN,B


In [161]:
# define function to reset the initial Time to zero,Lat=0,Lon=zero
import datetime
from datetime import datetime, timedelta 
def recode_time(df):
    fmt='%Y-%m-%dT%H:%M:%S'
    time_lst=df['BaseDateTime']
    lst=[]
    t0=time_lst.iloc[0]
    t0=datetime.strptime(t0,fmt)
    for time in time_lst:
        t1=datetime.strptime(time,fmt)
        t_diff=t1-t0
        t_diff=round(t_diff.total_seconds()/60,3)
        lst.append(t_diff)
    df['BaseDateTime']=lst
    origin_lat=df1['LAT'].iloc[0]
    origin_log=df1['LON'].iloc[0]
    Lat=df['LAT']-origin_lat
    Lon=df['LON']-origin_log
    df['LAT']=Lat
    df['LON']=Lon
    return df

In [162]:
df_recode=recode_time(df1)

In [164]:
df_recode.head()
df_recode.describe()

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselType,Status,Length,Width,Draft,Cargo
count,128.0,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.0,0.0,128.0,128.0,0.0,0.0
mean,368066040.0,164.197586,-0.200121,-0.167626,8.090625,-3.281250,205.656250,30.0,NaN,30.0,9.0,NaN,NaN
std,0.0,200.036044,0.208197,0.088719,2.216489,175.142592,38.463489,0.0,NaN,0.0,0.0,NaN,NaN
min,368066040.0,0.000000,-0.920480,-0.255100,0.400000,-204.000000,22.000000,30.0,NaN,30.0,9.0,NaN,NaN
25%,368066040.0,66.042000,-0.279635,-0.241353,8.250000,-170.625000,174.000000,30.0,NaN,30.0,9.0,NaN,NaN
50%,368066040.0,125.700000,-0.135885,-0.218060,9.050000,-118.800000,207.000000,30.0,NaN,30.0,9.0,NaN,NaN
75%,368066040.0,185.167000,-0.057720,-0.086970,9.425000,173.300000,238.000000,30.0,NaN,30.0,9.0,NaN,NaN
max,368066040.0,1031.300000,0.000000,0.000000,10.100000,204.000000,310.000000,30.0,NaN,30.0,9.0,NaN,NaN


In [195]:
# Plot the 3D route of this vessel
import plotly.graph_objects as go
layout = go.Layout(
    title = 'Vessel Route',
    yaxis = dict(title = 'Longitude'),
    xaxis = dict(title = 'Latitude'))

fig=go.FigureWidget(data=go.Scatter3d(
    x=df_recode['LAT'],
    y=df_recode['LON'],
    z=df_recode['BaseDateTime'],
    marker=dict(size=4,
                color=df_recode['BaseDateTime'],
                colorscale='Viridis'),
    line=dict(color='darkblue',
              width=2
             ),
),
              layout=layout)
fig.update_layout(
    width=800,
    height=700,
    autosize=False,
    scene=dict(
        camera=dict(
            up=dict(
                x=0,
                y=0,
                z=0
            ),
            eye=dict(
                x=0,
                y=1,
                z=1,
            )
        ),
        aspectratio = dict( x=1, y=1, z=0.7 ),
        aspectmode = 'manual'
    ),
)
fig

FigureWidget({
    'data': [{'line': {'color': 'darkblue', 'width': 2},
              'marker': {'color': arra…

FigureWidget({
    'data': [{'type': 'bar', 'uid': '9dfac039-981a-4657-9d2e-32501301fd20', 'y': [2, 3, 1]}],
 …